<a href="https://colab.research.google.com/github/GenericP3rson/WikiGraph/blob/main/WikiGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WikiGraph

## Schema Creation

In [11]:
!pip install pyTigerGraph[gds]

import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host = "https://wikigraph.i.tgcloud.io/")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
conn.gsql('''
USE GLOBAL

CREATE VERTEX Doc(PRIMARY_ID id STRING, title STRING, content STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"
CREATE VERTEX Entity(PRIMARY_ID entity STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"

CREATE DIRECTED EDGE LINKS_TO(FROM Doc, TO Doc, weight DOUBLE)
CREATE UNDIRECTED EDGE DOC_ENTITY(FROM Doc, TO Entity, weight DOUBLE)

CREATE GRAPH WikiGraph(Doc, Entity, LINKS_TO, DOC_ENTITY)
''')

'Successfully created vertex types: [Doc].\nSuccessfully created vertex types: [Entity].\nSuccessfully created edge types: [LINKS_TO].\nSuccessfully created edge types: [DOC_ENTITY].\nStopping GPE GSE RESTPP\nSuccessfully stopped GPE GSE RESTPP in 25.757 seconds\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.097 seconds\nThe graph WikiGraph is created.'

In [13]:
conn.graphname = "WikiGraph"
conn.apiToken = conn.getToken(conn.createSecret())

## Load Data

In [ ]:
!pip install yake

In [ ]:
import requests
from bs4 import BeautifulSoup
import yake
from collections import Counter

# Text cleaning, entity extraction, sentiment analysis, summarisation
# Graph structures

seen = []

def scrape(input_link):

  print(input_link)

  response = requests.get(
    url = input_link,
  )
  soup = BeautifulSoup(response.content, 'html.parser')

  first_paragraph = soup.find('p')

  if first_paragraph == None:
    return 

  while first_paragraph == None or first_paragraph.get_text().strip() == "":
    first_paragraph = first_paragraph.find_next_sibling('p')

  conn.upsertVertex("Doc", input_link, attributes={"id": input_link, "title": soup.find("h1").get_text(), "content": first_paragraph.get_text()})

  text = ("\n".join([''.join(i.get_text().split('\n')) for i in soup.find_all("p")]))

  kw_extractor = yake.KeywordExtractor()
  keywords = kw_extractor.extract_keywords(text)
  
  for word, score in keywords:
    conn.upsertVertex("Entity", word, attributes={"entity": word})
    conn.upsertEdge("Doc", input_link, "DOC_ENTITY", "Entity", word, {"weight": float(1-score)})

  seen.append(input_link)

  links = [f"https://en.wikipedia.org{link['href'].split('#')[0]}" for link in soup.find_all("a") if link.has_attr("href") and link["href"][:6] == "/wiki/" and not ":" in link["href"]]
  tot_links = len(links)

  count = Counter(links)
  for link in links:
    conn.upsertEdge("Doc", input_link, "LINKS_TO", "Doc", link, {"weight": float(count[link]/tot_links)})
    if link not in seen:
      scrape(link)

scrape("https://en.wikipedia.org/wiki/Graph_database")

## Queries

In [16]:
f = conn.gds.featurizer()

f.installAlgorithm("tg_pagerank_wt")
f.installAlgorithm("tg_article_rank")

Installing and optimizing the queries, it might take a minute
Installing and optimizing the queries, it might take a minute


'tg_article_rank'